In [ ]:
! pip install datasets fuzzywuzzy

Download `dataset.py` and `utils.py` from the repository by original paper authors [here](https://github.com/tzshi/squall/tree/main/model)

In [ ]:
import torch
from dataset import load_dataset, em_process

In [ ]:
DATASET_PATH = "squall.json"

#### Load and explore SQUALL dataset

In [ ]:
train_data = load_dataset(DATASET_PATH)

In [ ]:
train_data[0]

{'nt': 'nt-2254',
 'columns': [['constituency', ['constituency'], ['number'], 'number', '1'],
  ['region', ['region'], [], 'text', 'arica and parinacota\ntarapaca'],
  ['name',
   ['name'],
   ['first', 'second'],
   'text(text)',
   'jose antonio gomez urrutia\n(vice-president, march 11, 2010-march 15, 2011; march 20, 2013-present)'],
  ['party', ['party'], [], 'text', 'ps'],
  ['last elected', ['last', 'elected'], ['number'], 'number', '2009']],
 'nl': ['what',
  'is',
  'the',
  'difference',
  'in',
  'years',
  'between',
  'constiuency',
  '1',
  'and',
  '2',
  '?'],
 'nl_pos': ['WP',
  'VBD-AUX',
  'DT',
  'NN',
  'IN',
  'NNS',
  'IN',
  'NN',
  'CD',
  'CC',
  'CD',
  '.'],
 'nl_ner': ['O',
  'O',
  'O',
  'O',
  'O',
  'DURATION',
  'O',
  'O',
  'NUMBER',
  'O',
  'NUMBER',
  'O'],
 'nl_ralign': [['None', None],
  ['None', None],
  ['None', None],
  ['Keyword', ['diff', '']],
  ['None', None],
  ['Column', 'c5_number'],
  ['Keyword', ['diff', '']],
  ['Column', 'c1_number']

In [ ]:
# alignment annotation of NLQ with SQL typed labels

train_data[0]['nl_ralign']

[['None', None],
 ['None', None],
 ['None', None],
 ['Keyword', ['diff', '']],
 ['None', None],
 ['Column', 'c5_number'],
 ['Keyword', ['diff', '']],
 ['Column', 'c1_number'],
 ['Literal', None],
 ['Keyword', ['diff', '']],
 ['Literal', None],
 ['None', None]]

In [ ]:
# tokenized NLQ (Natural Language question)

train_data[0]['nl']

['what',
 'is',
 'the',
 'difference',
 'in',
 'years',
 'between',
 'constiuency',
 '1',
 'and',
 '2',
 '?']

In [ ]:
# columns of table headers for this NLQ

train_data[0]['columns']

[['constituency', ['constituency'], ['number'], 'number', '1'],
 ['region', ['region'], [], 'text', 'arica and parinacota\ntarapaca'],
 ['name',
  ['name'],
  ['first', 'second'],
  'text(text)',
  'jose antonio gomez urrutia\n(vice-president, march 11, 2010-march 15, 2011; march 20, 2013-present)'],
 ['party', ['party'], [], 'text', 'ps'],
 ['last elected', ['last', 'elected'], ['number'], 'number', '2009']]

In [ ]:
train_data[0]['columns'][0]

### Transform into BIO format for NER task

In [ ]:
SAMPLE_SIZE = 50
BIO_LABEL_MAP = {
    "None": "O",
    "Keyword": "O",
    "Column": "B-COLUMN",
    "Literal": "B-LITERAL"
}

#### Get tokenized nlq for 1st K rows

In [ ]:
nlq_list = []
for i in range(SAMPLE_SIZE):
  nlq_list.append(train_data[i]['nl'])

In [ ]:
len(nlq_list)

50

In [ ]:
nlq_list[:5]

[['what',
  'is',
  'the',
  'difference',
  'in',
  'years',
  'between',
  'constiuency',
  '1',
  'and',
  '2',
  '?'],
 ['what', 'is', 'the', 'total', 'number', 'of', 'constituencies', '?'],
 ['which', 'region', 'is', 'listed', 'below', 'atacama', '?'],
 ['which', 'party', 'did', 'jaime', 'quintana', 'belong', 'to', '?'],
 ['what', 'is', 'the', 'last', 'region', 'listed', 'on', 'the', 'table', '?']]

#### Get column list for the same samples above

In [ ]:
columns_list = []
for i in range(SAMPLE_SIZE):
  col_list_per_nlq = []
  for column_list in train_data[i]['columns']:
    col_list_per_nlq.append(column_list[0])
  columns_list.append(col_list_per_nlq)

In [ ]:
len(columns_list)

50

In [ ]:
columns_list[8:15]

[['constituency', 'region', 'name', 'party', 'last elected'],
 ['event', 'date', 'area', 'tornadoes', 'casualties', 'notes'],
 ['event', 'date', 'area', 'tornadoes', 'casualties', 'notes'],
 ['event', 'date', 'area', 'tornadoes', 'casualties', 'notes'],
 ['type', 'stage', 'cyclist', 'team', 'reason'],
 ['type', 'stage', 'cyclist', 'team', 'reason'],
 ['type', 'stage', 'cyclist', 'team', 'reason']]

#### Get SQL type labels for each mention in tokenized NLQ

In [ ]:
sql_labels = []

for i in range(SAMPLE_SIZE):
  sql_label_per_token = []
  for token in train_data[i]['nl_ralign']:
    sql_label_per_token.append(BIO_LABEL_MAP[token[0]])
  sql_labels.append(sql_label_per_token)

In [ ]:
len(sql_labels)

50

In [ ]:
sql_labels[:5]

[['O',
  'O',
  'O',
  'O',
  'O',
  'B-COLUMN',
  'O',
  'B-COLUMN',
  'B-LITERAL',
  'O',
  'B-LITERAL',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'B-COLUMN', 'O'],
 ['O', 'B-COLUMN', 'O', 'O', 'O', 'B-LITERAL', 'O'],
 ['O', 'B-COLUMN', 'O', 'B-LITERAL', 'B-LITERAL', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'B-COLUMN', 'O', 'O', 'O', 'O', 'O']]

#### Get question IDs

In [ ]:
qids = []
for i in range(SAMPLE_SIZE):
  qids.append(train_data[i]['nt'])

In [ ]:
len(qids)

50

In [ ]:
qids[:10]

['nt-2254',
 'nt-3125',
 'nt-3264',
 'nt-6446',
 'nt-7882',
 'nt-8692',
 'nt-9126',
 'nt-9181',
 'nt-12427',
 'nt-525']

### Convert sample to datasets instance for model training

In [ ]:
import pandas as pd

from datasets import load_dataset, Dataset, DatasetDict
from datasets import Features, ClassLabel, Sequence, Value

In [ ]:
# Collect above samples into a dictionary of features

squall_dict = {
    "ner_tags": sql_labels,
    "nl": nlq_list,
    "nt": qids,
    "columns": columns_list
}

In [ ]:
# Define features - data type for each value for keys in above dictionary
squall_features = Features({
 "nl": Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 "columns": Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 "ner_tags": Sequence(feature=ClassLabel(names=['O', 'B-COLUMN', 'I-COLUMN',
                                                'B-LITERAL', 'I-LITERAL'],
                                              id=None), length=-1, id=None),
 "nt": Value(dtype='string', id=None)
})

In [ ]:
squall_dataset = Dataset.from_dict(
    mapping=squall_dict,
    features=squall_features,
    split='train'
)

In [ ]:
squall_dataset_df = DatasetDict()
squall_dataset_df['train'] = squall_dataset

In [ ]:
squall_dataset_df

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'nl', 'nt', 'columns'],
        num_rows: 50
    })
})

In [ ]:
squall_dataset_df['train'][0]

{'ner_tags': [0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 3, 0],
 'nl': ['what',
  'is',
  'the',
  'difference',
  'in',
  'years',
  'between',
  'constiuency',
  '1',
  'and',
  '2',
  '?'],
 'nt': 'nt-2254',
 'columns': ['constituency', 'region', 'name', 'party', 'last elected']}

Notice that the ner_tags are automatically converted to integers from their NER labels provided in the dictionary form. Defining the features with `Sequence` & `ClassLabel` instances enables the `dataset` to infer the data type & perform conversion wherever neccessary.

In [ ]:
# NER tags for a sample of 1st 5 cases (with NER tags)
squall_dict['ner_tags'][:5]

[['O',
  'O',
  'O',
  'O',
  'O',
  'B-COLUMN',
  'O',
  'B-COLUMN',
  'B-LITERAL',
  'O',
  'B-LITERAL',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'B-COLUMN', 'O'],
 ['O', 'B-COLUMN', 'O', 'O', 'O', 'B-LITERAL', 'O'],
 ['O', 'B-COLUMN', 'O', 'B-LITERAL', 'B-LITERAL', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'B-COLUMN', 'O', 'O', 'O', 'O', 'O']]

In [ ]:
# Save this dataset to disk
squall_dataset_df.save_to_disk("squall_sample_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

### References
* SQUALL paper - https://arxiv.org/pdf/2010.11246
* https://github.com/tzshi/squall
* https://huggingface.co/docs/datasets/en/about_dataset_features
* https://huggingface.co/docs/datasets/v2.19.0/en/package_reference/main_classes#datasets.Sequence
* https://huggingface.co/docs/datasets/v2.19.0/en/package_reference/main_classes#datasets.Dataset